# __Toronto Neighborhood__

## Part 1

In [1]:
!pip install bs4
!pip install lxml

In [2]:
import pandas as pd
import requests
from lxml import html
from bs4 import BeautifulSoup

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content, "lxml")
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df=pd.DataFrame(df[0])

  
  
Deleting the boroughs that are not assigned

In [4]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)




Current Postal code list on Wikipedia page is already rouped.  There is no need to set a neighborhood name as the same as the borough. We have to replace the slash with the comma

In [5]:
df = df.replace({'/': ','}, regex=True)

In [6]:
PCdf=df.sort_values(by=['Postal code']).rename(columns={'Postal code': 'Postal Code'})
PCdf.reset_index(inplace=True)
PCdf.drop('index',axis=1,inplace=True)

In [7]:
PCdf

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [8]:
print(PCdf.shape)

(103, 3)



-----------------------------------------------------------------------
## Part 2

In [9]:
coord=pd.read_csv('Geospatial_Coordinates.csv') 

In [10]:
PCdf=PCdf.merge(coord,on='Postal Code')

In [11]:
PCdf

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


-------------------------------------------------------------
## Part 3

In [12]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
import json 
import requests 
from pandas.io.json import json_normalize 

In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [14]:

# Toronto map using latitude and longitude values
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Let's add markers to map
for lat, lng, borough, neighborhood in zip(PCdf['Latitude'], PCdf['Longitude'], PCdf['Borough'], PCdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#aacc31',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

Let's now look at the DownTown Toronto

In [15]:
Downtown_data = PCdf[PCdf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town , Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East , Union Station , Toronto Is...",43.640816,-79.381752


Let's see the Downtown location

In [16]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
dt_location = geolocator.geocode(address)
dt_latitude = dt_location.latitude
dt_longitude = dt_location.longitude
print('The geographical coordinate of Downtown Toronto are {}, {}.'.format(dt_latitude, dt_longitude))

The geographical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


Now I would like to get a map of Downtown Toronto neighborhoods

In [17]:
# Toronto map using latitude and longitude values
Downtown_map = folium.Map(location=[dt_latitude, dt_longitude], zoom_start=10)

# Let's add markers to map
for lat, lng, borough, neighborhood in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['Borough'], Downtown_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#aacc31',
        fill_opacity=0.7,
        parse_html=False).add_to(Downtown_map)  
    
Downtown_map

Let's explore the Dowtown Toronto with Foursquare information. I am interested in venues locate in "Central Bay Street" neighborhood.

First of all I need to create my credentials

In [18]:
CLIENT_ID = 'QDXT4GZASN3ZBFYCBB5V41BDYMB5P4MWOKKJ2RFY4ZF3FBFX'
CLIENT_SECRET = '31QLV23QJ5Z1DPUUAQC54CR1KL0KTBNBFF1EZTX0XNFQMSCL'
VERSION = '20200417' # Foursquare API version


In [19]:
# Church and Wellesley coordinates are
Central_Bay_Street_lat= Downtown_data.loc[7,'Latitude'] 
Central_Bay_Street_long= Downtown_data.loc[7,'Longitude']

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Central_Bay_Street_lat, 
    Central_Bay_Street_long, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=QDXT4GZASN3ZBFYCBB5V41BDYMB5P4MWOKKJ2RFY4ZF3FBFX&client_secret=31QLV23QJ5Z1DPUUAQC54CR1KL0KTBNBFF1EZTX0XNFQMSCL&v=20200417&ll=43.6579524,-79.3873826&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9cd7120de0d90028b40171'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 62,
  'suggestedBounds': {'ne': {'lat': 43.6624524045, 'lng': -79.38117421839567},
   'sw': {'lat': 43.6534523955, 'lng': -79.39359098160432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
        'label

Let's extract the information about venues in Central Bay Street with this function

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338


In [23]:
nearby_venues

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
...,...,...,...,...
57,Second Cup,Coffee Shop,43.660722,-79.386308
58,Pastacceria,Italian Restaurant,43.660020,-79.388188
59,Queen's Park Subway Station,Metro Station,43.660006,-79.390272
60,Mo'Ramyun,Korean Restaurant,43.656148,-79.392282


In [24]:
print('{} venues were returned by Foursquare in Central Bay Street.'.format(nearby_venues.shape[0]))

62 venues were returned by Foursquare in Central Bay Street.


Let's see the venues in the entire Downtown Toronto

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
Downtown_venues = getNearbyVenues(names=Downtown_data['Neighborhood'],
                                   latitudes=Downtown_data['Latitude'],
                                   longitudes=Downtown_data['Longitude']
                                  )

Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Queen's Park , Ontario Provincial Government


In [27]:
print(Downtown_venues.shape)
Downtown_venues.head()

(1215, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town , Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


### Let's see venues for each neighborhhod

In [28]:
Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",15,15,15,15,15,15
Central Bay Street,62,62,62,62,62,62
Christie,18,18,18,18,18,18
Church and Wellesley,71,71,71,71,71,71
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
"First Canadian Place , Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East , Union Station , Toronto Islands",100,100,100,100,100,100


How many unique categories are there?

In [29]:
print('There are {} unique categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 204 unique categories.


# Let's see what kind of venue is in each Neighborhood

In [30]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")
Downtown_onehot.drop(columns=['Neighborhood'], inplace=True)
# add neighborhood column back to dataframe
Downtown_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_venues.columns[0]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]
Downtown_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"St. James Town , Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### How about the frequency and the occurence of each category? Let's the mean

In [31]:
Downtown_grouped = Downtown_onehot.groupby(['Neighborhood']).mean().reset_index()
Downtown_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.018182,...,0.000000,0.000000,0.00,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower , King and Spadina , Railway Lands , ...",0.066667,0.066667,0.066667,0.133333,0.2,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.016129,0.000000,0.000000,0.000000,0.000000,0.016129
3,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.0,0.014085,0.000000,0.00,0.000000,...,0.014085,0.014085,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.028169
5,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.040000,0.000000,0.00,0.010000,...,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
6,"First Canadian Place , Underground city",0.000000,0.000000,0.000000,0.000000,0.0,0.030000,0.000000,0.00,0.010000,...,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.0,0.010000,0.000000,0.00,0.010000,...,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000
8,"Harbourfront East , Union Station , Toronto Is...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.05,0.010000,...,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Kensington Market , Chinatown , Grange Park",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.00,0.035088,0.000000,0.052632,0.017544,0.000000,0.000000


#### Let's see the top 5 most common venues in each venue

In [32]:
num_top_venues = 5

for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1            Beer Bar  0.04
2              Bakery  0.04
3  Italian Restaurant  0.04
4         Cheese Shop  0.04


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
             venue  freq
0  Airport Service  0.20
1   Airport Lounge  0.13
2          Airport  0.07
3              Bar  0.07
4      Coffee Shop  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.21
1  Italian Restaurant  0.08
2                Café  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03


----Christie----
                venue  freq
0       Grocery Store  0.22
1                Café  0.17
2                Park  0.11
3         Coffee Shop  0.06
4  Athletics & Sports  0.06


----Church and Wellesley----
                 venue  freq
0     Sushi Restaurant  0.07
1          Coffee Shop  0.07
2  Japanese Restaurant  0.06
3       

#### Let's this info in dataframe

this function is sorting the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
import numpy as np

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cheese Shop,Beer Bar,Farmers Market,Bakery,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
1,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport,Boutique,Rental Car Location,Coffee Shop,Boat or Ferry,Bar,Harbor / Marina,Sculpture Garden
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Japanese Restaurant,Fried Chicken Joint,Ice Cream Shop,Burger Joint,Bubble Tea Shop
3,Christie,Grocery Store,Café,Park,Gas Station,Restaurant,Italian Restaurant,Diner,Candy Store,Athletics & Sports,Baby Store
4,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Mediterranean Restaurant,Men's Store,Burger Joint,Hotel,Pub,Yoga Studio


<a id='item4'></a>

## Clustering the Neighborhoods with k-means

In [36]:
from sklearn.cluster import KMeans

In [37]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [38]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = Downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"St. James Town , Cabbagetown",43.667967,-79.367675,0,Park,Coffee Shop,Chinese Restaurant,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Gym / Fitness Center
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Mediterranean Restaurant,Men's Store,Burger Joint,Hotel,Pub,Yoga Studio
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Restaurant,Breakfast Spot,Theater,Café,Event Space,Performing Arts Venue
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Bookstore,Fast Food Restaurant


Finally, let's visualize the resulting clusters

In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Now the analysis

#### Cluster 1

In [41]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Park,Coffee Shop,Chinese Restaurant,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Gym / Fitness Center
2,Downtown Toronto,0,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Restaurant,Mediterranean Restaurant,Men's Store,Burger Joint,Hotel,Pub,Yoga Studio
3,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Restaurant,Breakfast Spot,Theater,Café,Event Space,Performing Arts Venue
4,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Bookstore,Fast Food Restaurant
5,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Italian Restaurant,Hotel,Clothing Store,Gym,Lingerie Store
6,Downtown Toronto,0,Coffee Shop,Café,Cheese Shop,Beer Bar,Farmers Market,Bakery,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Clothing Store,American Restaurant,Thai Restaurant,Gym,Concert Hall
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop
10,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,Seafood Restaurant,American Restaurant,Gastropub,Beer Bar
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega


#### Cluster 2

In [42]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 3

In [43]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Gas Station,Restaurant,Italian Restaurant,Diner,Candy Store,Athletics & Sports,Baby Store


#### Cluster 4

In [44]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Service,Airport Lounge,Airport,Boutique,Rental Car Location,Coffee Shop,Boat or Ferry,Bar,Harbor / Marina,Sculpture Garden


#### Cluster 5

In [45]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Japanese Restaurant,Fried Chicken Joint,Ice Cream Shop,Burger Joint,Bubble Tea Shop
18,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Diner,Creperie,Sandwich Place,Burger Joint,Burrito Place,Café,Park,College Auditorium


Thanks!